In [2]:
import torch
import torch.distributions as distributions
import torch.nn as nn
import torch.nn.functional as f
import torchvision
from torch.optim import Adam
from torch.utils.data import SubsetRandomSampler, DataLoader
from scipy import interpolate
import numpy as np

data = torchvision.datasets.MNIST(".", download=True, train=True)
new_data = torchvision.datasets.MNIST(".", download=True, train=False, transform=None)

use_cuda = torch.cuda.is_available()
device = torch.device('cuda' if use_cuda else 'cpu')

3.7.4 (default, Aug  9 2019, 18:34:13) [MSC v.1915 64 bit (AMD64)]


In [3]:
from matplotlib.pyplot import imshow
images = []
for i, (image, label) in enumerate(data):
    if i > 5000:
        break
    image_mod = np.array(image) 
    image_x = 28
    image_y = 28
    hole_size_x = 10
    hole_size_y = 10
    x = np.zeros((image_x*image_y,))
    y = np.zeros((image_x*image_y,))
    z = np.zeros((image_x*image_y,))
    counter = 0
    for i, row in enumerate(image_mod):
        for j, a in enumerate(row):
            x[counter] = j
            y[counter] = i
            z[counter] = a
            counter += 1    
    imshow(image_mod)
    img_center_x = image_x/2
    img_center_y = image_y/2
    hole_beg_x = img_center_x - hole_size_x/2
    hole_end_x = img_center_x + hole_size_x/2
    hole_beg_y = img_center_y - hole_size_y/2
    hole_end_y = img_center_y + hole_size_y/2
    mask = []
    #remove center rectangle
    for a in range(image_x*image_y):
        if not hole_beg_x < x[a] < hole_end_x or not hole_beg_y < y[a] < hole_end_x:
                mask.append(a)
    x = x[mask]
    y = y[mask]
    z = z[mask]
    # move points to fill hole
    x_old = np.copy(x)
    y_old = np.copy(y)
    for a in range(len(x)):
        if hole_beg_x <= x[a] <= hole_end_x or hole_beg_y <= y[a] <= hole_end_x:
            if x[a] > y[a] and (image_x - x[a]) > y[a]:
                y[a] *= (image_x - hole_size_x) / (image_x - abs(image_x - 2*x[a]))
                if np.isinf(y[a]):
                    print(a)
            elif x[a] < y[a] and (image_x - x[a]) < y[a]:
                x[a] *= (image_y - hole_size_y) / (image_y - abs(image_y - 2*y[a]))
            elif x[a] > y[a] and (image_x - x[a]) < y[a]:
                y[a] = (y[a] - image_y) * ((image_x - hole_size_x) / (image_x - abs(image_x - 2*x[a]))) + image_y
                if np.isinf(y[a]):
                    print(a)
            elif x[a] < y[a] and (image_x - x[a]) > y[a]:
                x[a] = (x[a] - image_x) * (image_y - hole_size_y) / (image_y - abs(image_y - 2*y[a])) + image_x
    x_new = np.arange(0,28,1)
    y_new = np.arange(0,28,1)
    x_new, y_new = np.meshgrid(x_new, y_new)
    z_new = interpolate.griddata((x, y), z, (x_new, y_new), method='linear')
    x_2 = interpolate.griddata((x, y), x_old, (x_new, y_new), method='linear')
    y_2 = interpolate.griddata((x, y), y_old, (x_new, y_new), method='linear')
    z_new[z_new<0]=0
    images.append(torch.from_numpy(np.stack([z_new, x_2, y_2])))

In [4]:
batch_size: int = 20
epoch: int = 10
lr: float = 0.01
test_split: float = 0.2

random_seed: int = 10
shuffle_dataset: bool = True

dataset_size: int = len(images)
indices: list = list(range(dataset_size))
split: int = int(np.floor(test_split * dataset_size))
if shuffle_dataset:
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, test_indices = indices[split:], indices[:split]

train_sampler: SubsetRandomSampler = SubsetRandomSampler(train_indices)
test_sampler: SubsetRandomSampler = SubsetRandomSampler(test_indices)

train_loader: DataLoader = DataLoader(images, batch_size=batch_size, sampler=train_sampler)
test_loader: DataLoader = DataLoader(images, batch_size=batch_size, sampler=test_sampler)

In [5]:
k = 3
l = 1
n = 10*10

class Network(torch.nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.conv1 = nn.Conv2d(3, 30, kernel_size=5)
        self.conv2 = nn.Conv2d(30, 60, kernel_size=5)
        # self.conv3 = nn.Conv2d(60, 90, kernel_size=5)
        self.linear1 = nn.Linear(320*3, 1256)
        self.linear2 = nn.Linear(1256, k + k * n + n * l * k + k * n)

    def forward(self, x: torch.Tensor):
        x = f.relu(f.max_pool2d(self.conv1(x), kernel_size=2))
        x = f.relu(f.max_pool2d(self.conv2(x), kernel_size=2))
        # x = f.relu(f.max_pool2d(self.conv3(x), kernel_size=2))
        x = x.view(-1, 320*3)
        x = f.relu(self.linear1(x))
        return self.linear2(x)


model: Network = Network().double()
optimizer = Adam(model.parameters(), lr)

In [14]:
epoch = 30
model = model.to('cpu')
def loss_function(x: torch.Tensor, input):
    x = x.view((len(input), -1))
    sum = torch.tensor(0).double().to(device)
    sum.requires_grad = True
    for i in range(len(input)):
        p:torch.Tensor = x[i][:k]
        p_tuple = torch.split(p, 1, 0)
        m:torch.Tensor = x[i][k:k+k*n].view(-1, k)
        m_tuple = torch.split(m, 1, 1)
        A:torch.Tensor = x[i][k+k*n:k+k*n+n*l*k].view(-1, l, k)
        A_tuple = torch.split(A, 1, 2)
        d:torch.Tensor = x[i][k+k*n+n*l*k:].view(-1,k)
        d_tuple = torch.split(d, 1, 1)
        dists = []
        for p, m, A, d in zip(p_tuple, m_tuple, A_tuple, d_tuple):
            dists.append((p, distributions.lowrank_multivariate_normal.LowRankMultivariateNormal(m.view(-1), A.view(n*l, l), torch.abs(d).view(-1))))
        for p, dist in dists:
            sum.add(-p.log().add(-dist.log_prob(input[i][0][9:19,9:19].contiguous().view(-1))))
    return sum

for e in range(epoch):
    for i, x in enumerate(train_loader):
        # x = x.to(device)
        optimizer.zero_grad()
        result = model(x.double())
        loss = loss_function(result, x)
        loss.backward()
    print("Epoch {} completed".format(e))


Epoch 0 completed
Epoch 1 completed
Epoch 2 completed
Epoch 3 completed
Epoch 4 completed
Epoch 5 completed
Epoch 6 completed
Epoch 7 completed
Epoch 8 completed
Epoch 9 completed
Epoch 10 completed
Epoch 11 completed
Epoch 12 completed
Epoch 13 completed
Epoch 14 completed
Epoch 15 completed
Epoch 16 completed
Epoch 17 completed
Epoch 18 completed
Epoch 19 completed
Epoch 20 completed
Epoch 21 completed
Epoch 22 completed
Epoch 23 completed
Epoch 24 completed
Epoch 25 completed
Epoch 26 completed
Epoch 27 completed
Epoch 28 completed
Epoch 29 completed


In [15]:
model(images[0].unsqueeze(0))[0][k:k+k*n].view(-1, k).split(1, 1)

(tensor([[-4.5391],
         [ 0.0698],
         [ 2.7748],
         [-1.6213],
         [-1.1211],
         [-6.4719],
         [-2.3872],
         [-5.1822],
         [-5.0768],
         [-3.7323],
         [ 7.9998],
         [-3.8013],
         [-3.0617],
         [ 2.2454],
         [-0.0831],
         [-1.2211],
         [-2.0102],
         [ 1.1770],
         [ 2.1792],
         [ 0.0340],
         [-7.9769],
         [-0.3310],
         [-1.5029],
         [ 5.0349],
         [-7.2467],
         [-4.3992],
         [ 3.3885],
         [ 1.3320],
         [-4.0132],
         [ 0.9017],
         [-4.3440],
         [-2.4263],
         [ 5.3133],
         [ 0.1555],
         [-6.6998],
         [-0.6416],
         [ 1.1528],
         [-1.5976],
         [-9.2596],
         [-0.9579],
         [ 5.1827],
         [-0.0495],
         [ 4.8090],
         [-7.9491],
         [-1.3177],
         [-3.9665],
         [-1.5022],
         [-6.2682],
         [ 4.2421],
         [-2.1748],


In [33]:
images[0][0][9:19,9:19].contiguous().view(-1)


tensor([156.0000,  56.3158, 101.1200, 111.4902,  53.5044,  16.0941,  24.4018,
         87.0824,   1.6431,   0.0000,   0.0000,   0.0000,  17.1452,  48.5970,
          0.3949,  13.4965,  52.4713,   0.0000,   0.0000,   0.0000,   0.0000,
          0.0000,   0.0000,   0.0000,   2.5913,   7.3421,   0.0000,   0.0000,
          0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
          0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
          0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
          0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
          0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
          0.0000,   0.0000,   0.0000,   5.6483,  42.3625,  79.0766,  68.3778,
          0.0000,   0.0000,   0.0000,   0.0000,   0.0000,  11.4201,  57.1007,
         87.7500, 126.8148, 156.9074,   0.0000,   0.0000,   0.0000,   0.0000,
         30.1978,  80.3333, 107.1736, 129.5625, 169.3750, 209.18